In [2]:
import requests
from bs4  import BeautifulSoup
import pandas as pd
import numpy as np

from time import sleep
from random import randint
from warnings import warn
from time import time

## Restaurant

In [6]:
#  1 webscraping Hotels
# Monitoring of the loop
start_time = time()
request = 0

# Dictionary for storing web content
soup_r = {}
for start_num in range(0, 240, 10):

    # Make a request
    url = f" https://www.yelp.com/search?find_desc=Restaurants&find_loc=london&start={start_num}"
    response = requests.get(url)

    # Pause the loop
    sleep(randint(8,15))
    print(response)
    page = BeautifulSoup(response.content, 'html.parser')
    soup_r[start_num] = page


    # Monitor the requests
    request += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(request, request/elapsed_time))
    #clear_output(wait = True)

    # Throw a warning if request wasn't successful
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(request, response.status_code))

<Response [200]>
Request:1; Frequency: 0.05400274302205798 requests/s
<Response [200]>
Request:2; Frequency: 0.06537721508885296 requests/s
<Response [200]>
Request:3; Frequency: 0.06889207906700058 requests/s
<Response [200]>
Request:4; Frequency: 0.06439553989574175 requests/s
<Response [200]>
Request:5; Frequency: 0.06546216175008474 requests/s
<Response [200]>
Request:6; Frequency: 0.06189250629800498 requests/s
<Response [200]>
Request:7; Frequency: 0.06417809003090115 requests/s
<Response [200]>
Request:8; Frequency: 0.06297737903028289 requests/s
<Response [200]>
Request:9; Frequency: 0.06388302960352578 requests/s
<Response [200]>
Request:10; Frequency: 0.061926138815926464 requests/s
<Response [200]>
Request:11; Frequency: 0.06185275398691129 requests/s
<Response [200]>
Request:12; Frequency: 0.06186256823638517 requests/s
<Response [200]>
Request:13; Frequency: 0.06234797621922646 requests/s
<Response [200]>
Request:14; Frequency: 0.061465100536033636 requests/s
<Response [20

In [7]:
resto_soup = pd.DataFrame([soup_r]).squeeze()

### Name

In [8]:
name = []
for row in resto_soup:  
    result = row.select('.css-1egxyvc .css-1m051bw') 
    for tag in result:
        print(tag.text)
        name.append(tag.text)

Restaurant Gordon Ramsay
The Mayfair Chippy
Dishoom
The Fat Bear
Duck & Waffle Local
Honey & Co
Duck & Waffle
The Breakfast Club
Flat Iron
The Queens Arms
The Victoria
Padella
Circolo Popolare
Sketch
Hawksmoor Seven Dials
Abeno
Mother Mash
Granger & Co
Gordon Ramsay Street Pizza
The Golden Chippy
The Ivy
The Grazing Goat Marylebone
Ye Olde Cheshire Cheese
Dishoom
Fuckoffee
Lanzhou Noodle Bar
Barrafina
The Rum Kitchen
NOPI
Hide
The Churchill Arms
The Ledbury
Ottolenghi
Cecconi’s Mayfair
Eggslut
The Ivy Chelsea Garden
Belgo Centraal
Seoul Bakery
Jinjuu
Ffiona’s Restaurant
BAO - Soho
The Alchemist
Holborn Dining Room
Mestizo
The Barge House
Kiln
Belgo Centraal
The English Rose Cafe and Tea Shop
Berners Tavern
Burger & Lobster
39 Steps Coffee
The Palomar Restaurant
Yauatcha
Brasserie Zédel
Carmen Bar De Tapas
Drunch Regent’s Park
Rules
Arabesque
Regency Café
Dominique Ansel Treehouse
Pieminister
Koya Soho
Bradleys Restaurant
Ngon Ngon
Naru
Haidilao Hot Pot
Savoir Faire
Shoryu Ramen
London 

In [16]:
# pd.Series(name).value_counts()

### Ratings

In [9]:
rating = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in resto_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        rat = header_text.select_one('.overflow--hidden__09f24___ayzG')
        if rat is not None:
            rating.append(rat['aria-label'].split()[0])
        else:
            rating.append(np.nan)

In [18]:
# test
# for n, rat in zip(name, rating):
#     print(n, rat) 

### Reviews

In [10]:
reviews = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in resto_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        rev = header_text.select_one('.reviewCount__09f24__tnBk4')
        if rev is not None:
            reviews.append(rev.text)
        else:
            reviews.append(np.nan)

In [20]:
# test
# for n, rev in zip(name, reviews):
#     print(n, rev) 

### Price

In [11]:
price_range = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in resto_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        price = header_text.select_one('.css-1s7bx9e')
        if price is not None:
            price_range.append(price.text)
        else:
            price_range.append(np.nan)

In [23]:
# test
# for n, price in zip(name, price_range):
#     print(n, price)


### Neighbours

In [12]:
neig = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in resto_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        neigh = header_text.select_one('.css-dzq7l1 .css-chan6m')
        if neigh is not None:
            neig.append(neigh.text)
        else:
            neig.append(np.nan)

In [30]:
# neig

### Type

In [13]:
type_res = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in resto_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        type = header_text.select_one('.css-dzq7l1')
        if type is not None:
            types_output = type.text
            type_res.append(types_output.split('€')[0])
        else:
            type_res.append(np.nan)

In [56]:
# type_res

# Dataframe

In [14]:
restaurant_df = pd.DataFrame({'restaurant_names':name, 'restaurant_ratings':rating, 'restaurant_reviews': reviews, 'restaurant_price': price_range, 'restaurant_neighbourhoods': neig, 'restaurant_type': type_res})

In [15]:
# store data
restaurant_df.to_csv('restaurant_data.csv', index=False)

In [61]:
#restaurant_df.head()

In [114]:
#restaurant_df.info()

In [34]:
# Missing values
#restaurant_df.isnull().sum().sort_values(ascending=False)

In [32]:
# duplicates values
#restaurant_df[restaurant_df.duplicated()]

### Changing Price value

In [17]:
# Price range (price per person)
# Based on information from yelp and Quora:
# $= under $10. $$=11-30. $$$=31-60. $$$$= over $61
# Converting to pound

# 1 pound = 1.31 USD
Euro = (pd.Series([10, 11, 30, 31, 60, 61]) * 1.31).apply(lambda x: int(x))
#print(Euro)
mapper = {'£':'under 13', '££':'[14 - 39]', '£££':'[40 - 78]', '££££':'over 79'}
restaurant_df['restaurant_price'] = restaurant_df['restaurant_price'].map(mapper)

In [59]:
# Top 20 Most reviews

#restaurant_df.sort_values(by='Reviews', ascending=False).head(20)

In [14]:
# Rating and number of reviews
restaurant_df['restaurant_ratings'] = restaurant_df['restaurant_ratings'].apply(lambda x: float(x)) 
restaurant_df['restaurant_reviews'] = restaurant_df['restaurant_reviews'].apply(lambda x: float(x))

#restaurant_df.describe()

# Hotels

In [17]:
#  1 webscraping Hotels
# Monitoring of the loop
start_time = time()
request = 0

# Dictionary for storing web content
soup_r = {}
for start_num in range(0, 240, 10):

    # Make a request
    url = f"https://www.yelp.com/search?find_desc=Hotels&find_loc=london&start={start_num}"
    response = requests.get(url)

    # Pause the loop
    sleep(randint(8,15))
    print(response)
    page = BeautifulSoup(response.content, 'html.parser')
    soup_r[start_num] = page


    # Monitor the requests
    request += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(request, request/elapsed_time))
    #clear_output(wait = True)

    # Throw a warning if request wasn't successful
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(request, response.status_code))

<Response [200]>
Request:1; Frequency: 0.0905118790065166 requests/s
<Response [200]>
Request:2; Frequency: 0.07170895257713836 requests/s
<Response [200]>
Request:3; Frequency: 0.06886014306986411 requests/s
<Response [200]>
Request:4; Frequency: 0.07175111676928002 requests/s
<Response [200]>
Request:5; Frequency: 0.0729300743921173 requests/s
<Response [200]>
Request:6; Frequency: 0.07412134489470248 requests/s
<Response [200]>
Request:7; Frequency: 0.07484024786574182 requests/s
<Response [200]>
Request:8; Frequency: 0.07314828516044275 requests/s
<Response [200]>
Request:9; Frequency: 0.07056979519331553 requests/s
<Response [200]>
Request:10; Frequency: 0.07004031369527525 requests/s
<Response [200]>
Request:11; Frequency: 0.06969967482006237 requests/s
<Response [200]>
Request:12; Frequency: 0.06819459533511969 requests/s
<Response [200]>
Request:13; Frequency: 0.06683683446425881 requests/s
<Response [200]>
Request:14; Frequency: 0.06749697104352449 requests/s
<Response [200]>


In [18]:
hotel_soup = pd.DataFrame([soup_r]).squeeze()

### Name

In [19]:
name = []
for row in hotel_soup:  
    result = row.select('.css-1m051bw') 
    for tag in result:
        print(tag.text)
        name.append(tag.text)

In [75]:
#pd.Series(name).value_counts()

### Ratings

In [20]:
rating = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in hotel_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        rat = header_text.select_one('.overflow--hidden__09f24___ayzG')
        if rat is not None:
            rating.append(rat['aria-label'].split()[0])
        else:
            rating.append(np.nan)

In [74]:
# test
# for n, rat in zip(name, rating):
#     print(n, rat) 

### Reviews

In [21]:
reviews = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in hotel_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        rev = header_text.select_one('.reviewCount__09f24__tnBk4')
        if rev is not None:
            reviews.append(rev.text)
        else:
            reviews.append(np.nan)

In [73]:
# test
# for n, rev in zip(name, reviews):
#     print(n, rev) 

### Price

In [22]:
price_range = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in hotel_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        price = header_text.select_one('.css-1s7bx9e')
        if price is not None:
            price_range.append(price.text)
        else:
            price_range.append(np.nan)

In [72]:
# test
# for n, price in zip(name, price_range):
#     print(n, price)

### Neighbours

In [48]:
neig = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in resto_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        neigh = header_text.select_one('.css-dzq7l1 .css-chan6m')
        if neigh is not None:
            neig.append(neigh.text)
        else:
            neig.append(np.nan)

In [51]:
# neig

### Type

# Dataframe

In [49]:
hotel_df = pd.DataFrame({'Name':name, 'Rating':rating, 'Reviews': reviews, 'Price Range in £': price_range, 'Neighbourhoods': neig})

In [50]:
# store data
hotel_df.to_csv('hotels_data.csv', index=False)

In [81]:
#hotel_df.head()

In [113]:
#hotel_df.info()

### Changing price value

In [25]:
# 1 pound = 1.31 USD
Euro = (pd.Series([10, 11, 30, 31, 60, 61]) * 1.31).apply(lambda x: int(x))
#print(Euro)
mapper = {'£':'under 13', '££':'[14 - 39]', '£££':'[40 - 78]', '££££':'over 79'}
hotel_df['Price Range in £'] = hotel_df['Price Range in £'].map(mapper)

In [86]:
# Top 20 Most reviews

#hotel_df.sort_values(by='Reviews', ascending=False).head(20)

# Pubs

In [34]:
#  1 webscraping Hotels
# Monitoring of the loop
start_time = time()
request = 0

# Dictionary for storing web content
soup_r = {}
for start_num in range(0, 240, 10):

    # Make a request
    url = f"https://www.yelp.com/search?find_desc=Pubs&find_loc=london&start={start_num}"
    response = requests.get(url)

    # Pause the loop
    sleep(randint(8,15))
    print(response)
    page = BeautifulSoup(response.content, 'html.parser')
    soup_r[start_num] = page


    # Monitor the requests
    request += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(request, request/elapsed_time))
    #clear_output(wait = True)

    # Throw a warning if request wasn't successful
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(request, response.status_code))

<Response [200]>
Request:1; Frequency: 0.08907339946500796 requests/s
<Response [200]>
Request:2; Frequency: 0.07593062561911493 requests/s
<Response [200]>
Request:3; Frequency: 0.07044308591041991 requests/s
<Response [200]>
Request:4; Frequency: 0.07323655290613018 requests/s
<Response [200]>
Request:5; Frequency: 0.07098803240368465 requests/s
<Response [200]>
Request:6; Frequency: 0.07173331084461493 requests/s
<Response [200]>
Request:7; Frequency: 0.07466147304465691 requests/s
<Response [200]>
Request:8; Frequency: 0.07425982115484663 requests/s
<Response [200]>
Request:9; Frequency: 0.07574759601587132 requests/s
<Response [200]>
Request:10; Frequency: 0.0763184357940579 requests/s
<Response [200]>
Request:11; Frequency: 0.07681826343301434 requests/s
<Response [200]>
Request:12; Frequency: 0.07579676602851569 requests/s
<Response [200]>
Request:13; Frequency: 0.07440941026723989 requests/s
<Response [200]>
Request:14; Frequency: 0.07337044713073766 requests/s
<Response [200]>

In [35]:
pub_soup = pd.DataFrame([soup_r]).squeeze()

### Name

In [36]:
name = []
for row in pub_soup:  
    result = row.select('.css-1egxyvc .css-1m051bw') 
    for tag in result:
        #print(tag.text)
        name.append(tag.text)

In [112]:
#pd.Series(name).value_counts()

### Ratings

In [37]:
rating = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in pub_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        rat = header_text.select_one('.overflow--hidden__09f24___ayzG')
        if rat is not None:
            rating.append(rat['aria-label'].split()[0])
        else:
            rating.append(np.nan)

In [96]:
# test
# for n, rat in zip(name, rating):
#     print(n, rat) 

### Reviews

In [38]:
reviews = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in pub_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        rev = header_text.select_one('.reviewCount__09f24__tnBk4')
        if rev is not None:
            reviews.append(rev.text)
        else:
            reviews.append(np.nan)

In [99]:
# test
# for n, rev in zip(name, reviews):
#     print(n, rev) 

### Price

In [39]:
price_range = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in pub_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        price = header_text.select_one('.css-1s7bx9e')
        if price is not None:
            price_range.append(price.text)
        else:
            price_range.append(np.nan)

In [102]:
# test
# for n, price in zip(name, price_range):
#     print(n, price)

### Neighbours

In [40]:
neig = []
container_class = 'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'
for row in resto_soup:
    result = row.find_all(class_ = container_class)

    for tag in result:
        header_text = tag.div.div.div.nextSibling.div
        neigh = header_text.select_one('.css-dzq7l1 .css-chan6m')
        if neigh is not None:
            neig.append(neigh.text)
        else:
            neig.append(np.nan)

In [42]:
# neig

### Type

# Dataframe

In [43]:
pub_df = pd.DataFrame({'Name':name, 'Rating':rating, 'Reviews': reviews, 'Price Range in £': price_range, 'Neighbourhoods': neig})

In [44]:
# store data
pub_df.to_csv('pubs_data.csv', index=False)

In [107]:
#pub_df.head()

In [108]:
#pub_df.info()

### Changing price value

In [45]:
# 1 pound = 1.31 USD
Euro = (pd.Series([10, 11, 30, 31, 60, 61]) * 1.31).apply(lambda x: int(x))
#print(Euro)
mapper = {'£':'under 13', '££':'[14 - 39]', '£££':'[40 - 78]', '££££':'over 79'}
pub_df['Price Range in £'] = pub_df['Price Range in £'].map(mapper)

In [111]:
# Top 20 Most reviews

#pub_df.sort_values(by='Reviews', ascending=False).head(20)